## Distributed Word Vectors

### Preparing the data

In [2]:
import pandas as pd

train = pd.read_csv('data/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('data/testData.tsv', header=0, delimiter='\t', quoting=3)

# unlabeled data
unlabeled_train = pd.read_csv('data/unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [8]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [9]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [11]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probabl

### Training and Saving The Model

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# values for paramters
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

# Initialize and train the model
from gensim.models import word2vec

print('Training models...')
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, 
                          min_count= min_word_count, window= context, sample= downsampling)

# init_sims will make the model much more memory-efficient. Stop the training
model.init_sims(replace=True)

# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-12-05 13:55:29,936 : INFO : collecting all words and their counts
2019-12-05 13:55:29,939 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-05 13:55:30,086 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training models...


2019-12-05 13:55:30,197 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-12-05 13:55:30,314 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-12-05 13:55:30,420 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-12-05 13:55:30,553 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-12-05 13:55:30,705 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-12-05 13:55:30,826 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-12-05 13:55:30,944 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-12-05 13:55:31,082 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-12-05 13:55:31,249 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50

2019-12-05 13:55:38,949 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2019-12-05 13:55:39,054 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-12-05 13:55:39,153 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-12-05 13:55:39,257 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-12-05 13:55:39,360 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-12-05 13:55:39,466 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-12-05 13:55:39,527 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-12-05 13:55:39,529 : INFO : Loading a fresh vocabulary
2019-12-05 13:55:39,706 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019

2019-12-05 13:56:35,985 : INFO : EPOCH 2 - PROGRESS: at 80.15% examples, 416036 words/s, in_qsize 8, out_qsize 0
2019-12-05 13:56:36,993 : INFO : EPOCH 2 - PROGRESS: at 82.67% examples, 412217 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:56:38,017 : INFO : EPOCH 2 - PROGRESS: at 85.05% examples, 407916 words/s, in_qsize 8, out_qsize 0
2019-12-05 13:56:39,026 : INFO : EPOCH 2 - PROGRESS: at 88.13% examples, 407263 words/s, in_qsize 8, out_qsize 0
2019-12-05 13:56:40,027 : INFO : EPOCH 2 - PROGRESS: at 91.87% examples, 409763 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:56:41,033 : INFO : EPOCH 2 - PROGRESS: at 95.71% examples, 412024 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:56:42,045 : INFO : EPOCH 2 - PROGRESS: at 99.44% examples, 414314 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:56:42,149 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-05 13:56:42,161 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-05 13:56:42,168 : I

2019-12-05 13:57:41,599 : INFO : EPOCH 4 - PROGRESS: at 97.51% examples, 439532 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:57:42,243 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-05 13:57:42,258 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-05 13:57:42,288 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-05 13:57:42,312 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-05 13:57:42,313 : INFO : EPOCH - 4 : training on 17798082 raw words (12749559 effective words) took 29.0s, 439805 effective words/s
2019-12-05 13:57:43,347 : INFO : EPOCH 5 - PROGRESS: at 3.87% examples, 484776 words/s, in_qsize 7, out_qsize 0
2019-12-05 13:57:44,355 : INFO : EPOCH 5 - PROGRESS: at 7.83% examples, 490603 words/s, in_qsize 8, out_qsize 0
2019-12-05 13:57:45,370 : INFO : EPOCH 5 - PROGRESS: at 11.73% examples, 489239 words/s, in_qsize 8, out_qsize 0
2019-12-05 13:57:46,371 : INFO : EPOCH 5 - PRO

### Exploring Model Result

In [13]:
model.doesnt_match("man woman child kitchen".split())

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [14]:
model.doesnt_match("france england germany berlin".split())

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [15]:
model.doesnt_match("paris berlin london austria".split())

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [16]:
model.most_similar("man")

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6070415377616882),
 ('lady', 0.5974882245063782),
 ('lad', 0.5737225413322449),
 ('farmer', 0.5273995399475098),
 ('men', 0.5217175483703613),
 ('millionaire', 0.5215692520141602),
 ('person', 0.5166182518005371),
 ('guy', 0.5063775777816772),
 ('chap', 0.5058619379997253),
 ('monk', 0.5050084590911865)]

In [17]:
model.most_similar("queen")

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6734331846237183),
 ('bride', 0.6208693981170654),
 ('showgirl', 0.5959992408752441),
 ('belle', 0.5832359194755554),
 ('victoria', 0.5817513465881348),
 ('maid', 0.5764146447181702),
 ('goddess', 0.5736275911331177),
 ('regina', 0.5718240737915039),
 ('stepmother', 0.5707747936248779),
 ('aurora', 0.570489227771759)]

In [18]:
model.most_similar("awful")

/Users/ujjawalpathak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7674058675765991),
 ('horrible', 0.7270845174789429),
 ('dreadful', 0.7221974730491638),
 ('atrocious', 0.7163230180740356),
 ('abysmal', 0.7028617858886719),
 ('horrendous', 0.6942626237869263),
 ('horrid', 0.6679326295852661),
 ('lousy', 0.6571812033653259),
 ('appalling', 0.6511085033416748),
 ('amateurish', 0.6192115545272827)]

In [19]:
### End